<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/stock_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ref: https://www.simplilearn.com/tutorials/deep-learning-tutorial/rnn
# https://www.kaggle.com/datasets/akram24/google-stock-price-test


In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [10]:
path = "/gdrive/MyDrive/Colab_data_files/stock_price"

In [11]:
data_folder=pathlib.Path(path)

In [13]:
dataset_train = pd.read_csv(data_folder / "Google_Stock_Price_Train.csv")
dataset_test = pd.read_csv(data_folder / "Google_Stock_Price_Test.csv")

In [14]:
dataset_train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [16]:
train_data = dataset_train.iloc[:,1:2]
train_data.head()

,Open
0,325.25
1,331.27
2,329.83
3,328.34
4,322.04


In [20]:
train_data.shape

(1258, 1)

In [18]:
mm = MinMaxScaler(feature_range = (0,1))
training_set_scaled = mm.fit_transform(train_data)


In [27]:
X_train = []
y_train = []


In [28]:
for i in range(60,1258):
  X_train.append(training_set_scaled[i-60:i,0])
  y_train.append(training_set_scaled[i,0])

In [29]:
X_train , y_train = np.array(X_train), np.array(y_train)

In [30]:
X_train.shape

(1198, 60)

In [31]:
X_train= np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [32]:
X_train.shape

(1198, 60, 1)

In [59]:
model = tf.keras.Sequential()

In [60]:
model.add(tf.keras.layers.Input(shape = (X_train.shape[1], 1)))

In [61]:
model.add(tf.keras.layers.LSTM(units = 50, return_sequences = True))

In [62]:
model.add(tf.keras.layers.Dropout(0.2))

In [63]:
model.add(tf.keras.layers.LSTM(units = 50, return_sequences = True))

In [64]:
model.add(tf.keras.layers.Dropout(0.2))

In [65]:
model.add(tf.keras.layers.LSTM(units = 50))

In [66]:
model.add(tf.keras.layers.Dropout(0.1))

In [67]:
model.add(tf.keras.layers.Dense(1))

In [68]:
model.compile(optimizer = "adam", loss = "mse")

In [69]:
model.fit(X_train,y_train,batch_size = 32, epochs = 100)

Epoch 1/100
38/38 [==============================] - 11s 111ms/step - loss: 0.0301
Epoch 2/100
38/38 [==============================] - 5s 133ms/step - loss: 0.0037
Epoch 3/100
38/38 [==============================] - 4s 100ms/step - loss: 0.0037
Epoch 4/100
38/38 [==============================] - 3s 79ms/step - loss: 0.0036
Epoch 5/100
38/38 [==============================] - 3s 80ms/step - loss: 0.0030
Epoch 6/100
38/38 [==============================] - 4s 105ms/step - loss: 0.0032
Epoch 7/100
38/38 [==============================] - 4s 103ms/step - loss: 0.0032
Epoch 8/100
38/38 [==============================] - 3s 81ms/step - loss: 0.0034
Epoch 9/100
38/38 [==============================] - 3s 84ms/step - loss: 0.0029
Epoch 10/100
38/38 [==============================] - 4s 107ms/step - loss: 0.0031
Epoch 11/100
38/38 [==============================] - 4s 95ms/step - loss: 0.0031
Epoch 12/100
38/38 [==============================] - 3s 81ms/step - loss: 0.0026
Epoch 13/100
38/38

In [ ]:
test_data = dataset_test.iloc[:,1:2]

In [72]:
dataset_total=pd.concat([train_data['Open'],test_data['Open']],axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = mm.transform(inputs)
X_test = []
for i in range(60,80):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test, (X_test.shape[0], X_test.shape[1],1))
pred = model.predict(X_test)
pred = mm.invserse_transform(ored)

NameError: ignored